In [1]:
import argparse
import datetime
import os
import shutil
from shutil import copyfile
import __init__ as booger
import yaml
from tasks.semantic.modules.data_analysis import *
from pip._vendor.distlib.compat import raw_input

from tasks.semantic.modules.SalsaNextAdf import *
from tasks.semantic.modules.SalsaNext import *
#from tasks.semantic.modules.save_dataset_projected import *
import math
from decimal import Decimal
import numpy as np

import open3d as o3d
from open3d import JVisualizer

In [2]:
dataset = "/workspace/datadir/dataset"
log = "/workspace/datadir/visualization"

print("dataset", dataset)
# open arch config file
try:
    arch_file = "/workspace/dllearn/SalsaNext/salsanext.yml"
    print("Opening arch config file %s" % arch_file)
    ARCH = yaml.safe_load(open(arch_file, 'r'))
except Exception as e:
    print(e)
    print("Error opening arch yaml file.")
    quit()

# open data config file
try:
    data_cfg = "/workspace/dllearn/SalsaNext/train/tasks/semantic/config/labels/semantic-kitti.yaml"
    print("Opening data config file %s" % data_cfg)
    DATA = yaml.safe_load(open(data_cfg, 'r'))
except Exception as e:
    print(e)
    print("Error opening data yaml file.")
    quit()

# create trainer and start the training
dataanalysis = DataAnalysis(ARCH, DATA, dataset, log)

dataset /workspace/datadir/dataset
Opening arch config file /workspace/dllearn/SalsaNext/salsanext.yml
Opening data config file /workspace/dllearn/SalsaNext/train/tasks/semantic/config/labels/semantic-kitti.yaml
Sequences folder exists! Using sequences from /workspace/datadir/dataset/sequences
parsing seq 00
parsing seq 01
parsing seq 02
parsing seq 03
parsing seq 04
parsing seq 05
parsing seq 06
parsing seq 07
parsing seq 09
parsing seq 10
Using 19130 scans from sequences [0, 1, 2, 3, 4, 5, 6, 7, 9, 10]
Using 19130 poses
Sequences folder exists! Using sequences from /workspace/datadir/dataset/sequences
parsing seq 08
Using 4071 scans from sequences [8]
Using 4071 poses


In [3]:
#pcd = dataanalysis.datao3d()

In [4]:
#o3d.io.write_point_cloud(log + "/input.ply", pcd)

In [5]:
points0, pose0, scan_file0, sem_label0 = dataanalysis.getpointspose(0)
points1, pose1, scan_file1, sem_label1 = dataanalysis.getpointspose(70)

TypeError: open_scan() missing 3 required positional arguments: 'prefile', 'prepose', and 'curpose'

In [ ]:
print(scan_file0)
print(scan_file1)
print(pose0.reshape(3,4))
print(pose1.reshape(3,4))
print(points0.shape)
print(points1.shape)
print(sem_label0.shape)
print(sem_label1.shape)

In [ ]:
# def changecor(point):
#     temp = np.zeros(4, dtype=np.float)
#     temp[0] = -point[1]
#     temp[1] = -point[2]
#     temp[2] = point[0]
#     temp[3] = 1
#     return temp

# def TranformWorld(points, pose):
#     x = pose[0:4]
#     y = pose[4:8]
#     z = pose[8:12]
#     trans = np.vstack((x, y, z))
#     print(trans)
#     world_points = np.array([np.dot(trans, changecor(point)) for point in points])
#     return world_points

In [ ]:
def TranformWorld(points, pose):
    x = pose[0:4]
    y = pose[4:8]
    z = pose[8:12]
    temp = np.vstack((x, y, z))
    trans = np.zeros_like(temp)
    trans[:, 0] = temp[:, 2]
    trans[:, 1] = -temp[:, 0]
    trans[:, 2] = -temp[:, 1]
    trans[:, 3] = temp[:, 3]
    world_points = np.array([np.dot(trans, np.append(point, 1)) for point in points])
    return world_points

In [ ]:
# world_points0 = TranformWorld(points0, pose0)
# world_points1 = TranformWorld(points1, pose1)

In [ ]:
colorize = np.array([[0,0,0], [245, 150, 100], [245, 230, 100], [150, 60, 30], [180, 30, 80], [255, 0, 0], [30, 30, 255], [200, 40, 255], [90, 30, 150], [255, 0, 255], [255, 150, 255], [75, 0, 75], [75, 0, 175], [0, 200, 255], [50, 120, 255], [0, 175, 0], [0, 60, 135], [80, 240, 150], [150, 240, 255], [0, 0, 255]], dtype=np.float)

In [ ]:
N0, _ = points0.shape
N1, _ = points1.shape

In [ ]:
colors0 = np.array([colorize[label] / 255. for label in sem_label0])
colors1 = np.array([colorize[label] / 255. for label in sem_label1])

In [ ]:
# points = np.vstack((world_points0, world_points1))
# colors = np.vstack((colors0, colors1))

In [ ]:
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)
# pcd.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
#o3d.io.write_point_cloud(log + "/world.ply", pcd)

In [ ]:
# def changeTrCor(temp):
#     trans = np.zeros_like(temp)
#     trans[:, 0] = temp[:, 2]
#     trans[:, 1] = -temp[:, 0]
#     trans[:, 2] = -temp[:, 1]
#     trans[:, 3] = temp[:, 3]
    
#     return trans

def TransformC2C(points, pose_f, pose_t):
    trans_f = pose_f.reshape(3, 4)
    trans_t = pose_t.reshape(3, 4)
    trans_f = trans_f[:, [2, 0, 1, 3]]
    trans_f[:, 1] *= -1
    trans_f[:, 2] *= -1
    trans_t = trans_t[:, [2, 0, 1, 3]]
    trans_t[:, 1] *= -1
    trans_t[:, 2] *= -1
    
    world_points = np.array([np.dot(trans_f, np.append(point, 1)) for point in points])
    
    shift = -trans_t[:, 3]
    rot = np.linalg.inv(trans_t[:, 0:3])
    anocamera_points = np.array([np.dot(rot, point+shift) for point in world_points])
    return anocamera_points

In [ ]:
transpoints0 = TransformC2C(points0, pose0, pose1)

In [ ]:
points = np.vstack((transpoints0, points1))
colors = np.vstack((colors0, colors1))

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
o3d.io.write_point_cloud(log + "/trans.ply", pcd)

In [ ]:
np.unique(sem_label1)

In [ ]:
a = np.array([[0,1,2],[3,1,0],[2,1,3]])
print(a)
a_on = np.array([np.identity(4)[a_row] for a_row in a])
print(a_on)
print(a_on.shape)

In [ ]:
a_on[1,0]